In [1]:
import csv
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
import nltk.sentiment.vader
from nltk.stem import PorterStemmer
stemming = PorterStemmer()
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [2]:
filename_t = pd.read_csv('E:/256/Project/review.csv',delimiter = ',')

In [3]:
filename = filename_t.iloc[:100000,:]
filename = filename.dropna()

In [4]:
del filename['date']
filename.head()

,business_id,cool,funny,review_id,stars,text,useful,user_id
0,ujmEBvifdJM6h6RLv4wQIg,0,1,Q1sbwvVQXV2734tPgoKj4Q,1.0,Total bill for this horrible service? Over $8G...,6,hG7b0MtEbXx5QzbzE6C_VA
1,NZnhc2sEQy3RmzKTZnqtwQ,0,0,GJXCdrto3ASJOqKeVWPi6Q,5.0,I *adore* Travis at the Hard Rock's new Kelly ...,0,yXQM5uF2jS6es16SJzNHfg
2,WTqjgwHlXbSFevF32_DJVw,0,0,2TzJjDVDEuAW6MR5Vuc1ug,5.0,I have to say that this office really has it t...,3,n6-Gk65cPZL6Uz8qRm3NYw
3,ikCg8xy5JIg_NGPx-MSIDA,0,0,yi0R0Ugj_xUx_Nek0-_Qig,5.0,Went in for a lunch. Steak sandwich was delici...,0,dacAIZ6fTM6mqwW5uxkskg
4,b1b1eb3uo-w561D0ZfCEiQ,0,0,11a8sVPMUFtaC7_ABRkmtw,1.0,Today was my second out of three sessions I ha...,7,ssoyf2_x0EQMed6fgHeMyQ


In [5]:
filename = filename[['text','stars']]

In [6]:
filename['text']=filename['text'].str.lower()

In [7]:
def remove_punctuation(value):
    result = ""
    for c in value:
        # If char is not punctuation, add it to the result.
        if c not in string.punctuation:
            result += c
    return result
for i,d in enumerate(filename['text']):
    filename['text'][i] = remove_punctuation(d)

C:\Users\sampr\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [8]:
filename

,text,stars
0,total bill for this horrible service over 8gs ...,1.0
1,i adore travis at the hard rocks new kelly car...,5.0
2,i have to say that this office really has it t...,5.0
3,went in for a lunch steak sandwich was delicio...,5.0
4,today was my second out of three sessions i ha...,1.0
...,...,...
99995,i love the clothes in aritzia the service i ca...,3.0
99996,we went here to have a small dinner with famil...,1.0
99997,amazing our 2 boys never had this much fun so...,5.0
99998,wanted a short hair cut but didnt have a regul...,5.0


In [9]:
filename.stars = filename.stars.map({5:'positive',4:'positive',3:'positive',2:'negative',1:'negative'})

In [10]:
stop = stopwords.words('english')
filename['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

0        total bill horrible service 8gs crooks actuall...
1        adore travis hard rocks new kelly cardenas sal...
2        say office really together organized friendly ...
3        went lunch steak sandwich delicious caesar sal...
4        today second three sessions paid although firs...
                               ...                        
99995    love clothes aritzia service without one shopp...
99996    went small dinner family 6 us waiter recommend...
99997    amazing 2 boys never much fun much dolaser tag...
99998    wanted short hair cut didnt regular hairstylis...
99999    place rad dueling pianos happy hour till 8 3 w...
Name: text, Length: 100000, dtype: object

In [11]:
"""for x in filename['text']:
    ss = sid.polarity_scores(x)
    #print(x)
    for k in sorted(ss):
        print('{0}: {1}, '.format(k, ss[k]), end='')
    print()"""

"for x in filename['text']:\n    ss = sid.polarity_scores(x)\n    #print(x)\n    for k in sorted(ss):\n        print('{0}: {1}, '.format(k, ss[k]), end='')\n    print()"

In [12]:
filename

,text,stars
0,total bill for this horrible service over 8gs ...,negative
1,i adore travis at the hard rocks new kelly car...,positive
2,i have to say that this office really has it t...,positive
3,went in for a lunch steak sandwich was delicio...,positive
4,today was my second out of three sessions i ha...,negative
...,...,...
99995,i love the clothes in aritzia the service i ca...,positive
99996,we went here to have a small dinner with famil...,negative
99997,amazing our 2 boys never had this much fun so...,positive
99998,wanted a short hair cut but didnt have a regul...,positive


In [13]:
filename['label'] = filename['stars'].apply(lambda x: 0 if x=='negative' else 1)

In [14]:
filename['stars'] = filename['stars'].apply(lambda x: x.split())
filename.stars = filename.stars.apply(lambda x: [stemming.stem(i) for i in x])
X_train, X_test, y_train, y_test = train_test_split(filename['text'], filename['label'], random_state=1)
cv = CountVectorizer(strip_accents='ascii', token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b')
X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_cv, y_train)
predictions = naive_bayes.predict(X_test_cv)
print('Accuracy score: ', accuracy_score(y_test, predictions),'Precision score: ', precision_score(y_test, predictions),'Recall score: ', recall_score(y_test, predictions))

Accuracy score:  0.88016 Precision score:  0.9250674133997097 Recall score:  0.9200103145951521
